# Models NOT YET ADJUSTED 

Import libraries. 

In [2]:
# Import necessary libraries
import json
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, auc, precision_recall_curve, confusion_matrix
import pandas as pd
import logging
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE


## Load data 

In [30]:
merged_df = pd.read_csv('/Users/dionnespaltman/Desktop/V3/merged_df.csv', sep=',')

merged_df.drop('Unnamed: 0', axis=1, inplace=True)
merged_df.drop('Unnamed: 0.1', axis=1, inplace=True)

display(merged_df.head(5))

,ID,sum_12,sum_4567,sum_456,VVR_group,Condition,VVR_1,VVR_2,AU01_r__sum_values,AU01_r__variance,...,AU26_r__minimum,AU26_r__mean,AU26_r__mean_abs_change,AU45_r__sum_values,AU45_r__variance,AU45_r__standard_deviation,AU45_r__maximum,AU45_r__minimum,AU45_r__mean,AU45_r__mean_abs_change
0,23,24.0,37.0,27.0,0,2,13.0,11.0,4982.48,0.425041,...,0.00,0.633284,0.076328,9231.74,0.825039,0.908316,4.91,0.00,0.627753,0.133624
1,24,23.0,37.0,28.0,0,2,12.0,11.0,9390.23,0.448366,...,0.00,1.484701,0.125851,11887.00,0.634554,0.796589,5.00,0.00,0.436942,0.098134
2,25,28.0,44.0,33.0,1,2,16.0,12.0,6954.35,0.599805,...,0.00,0.862301,0.101969,9020.78,0.750701,0.866430,4.04,0.00,0.550652,0.085720
3,26,30.0,37.0,29.0,0,1,15.0,15.0,9707.43,0.873280,...,0.00,0.552359,0.069582,6585.31,0.609348,0.780607,4.90,0.00,0.371673,0.056287
4,27,22.0,39.0,31.0,1,2,11.0,11.0,21049.90,1.475421,...,-3.92,0.142027,0.386527,23027.73,1.160635,1.077328,5.04,-4.29,1.094318,0.231853


In [31]:
# Count the number of instances of people in VVR_group = 1 and VVR_group = 0
count_vvr_group = merged_df['VVR_group'].value_counts()

# Print the counts
print("Number of instances in VVR_group = 1:", count_vvr_group[1])
print("Number of instances in VVR_group = 0:", count_vvr_group[0])


Number of instances in VVR_group = 1: 26
Number of instances in VVR_group = 0: 85


In [ ]:
with open('/Users/dionnespaltman/Desktop/V3/columns_action_units.json', 'r') as f:
    columns_action_units = json.load(f)

print(len(columns_action_units))
# print(columns_action_units)

## Define models  

In [21]:
# dummy = make_pipeline(featurizer, DummyClassifier(strategy='most_frequent'))
# rf = make_pipeline(featurizer, RandomForestClassifier(n_estimators=200, n_jobs=-1, random_state=0))
# svm = make_pipeline(featurizer, SVC())
# multiclass_svm = make_pipeline(featurizer, SVC(decision_function_shape='ovr'))
# xgb = make_pipeline(featurizer, XGBClassifier())
# mlp = make_pipeline(featurizer, MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000))

# models = {
#     "Dummy": dummy,
#     "RandomForest": rf,
#     "SVM": svm,
#     "Multiclass SVM": multiclass_svm,
#     "XGBoost": xgb,
#     "MLP": mlp
# }


from sklearn.pipeline import make_pipeline
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier

dummy = make_pipeline(featurizer, DummyClassifier(strategy='most_frequent'))
rf = make_pipeline(featurizer, RandomForestClassifier(n_estimators=200, n_jobs=-1, random_state=0))
svm = make_pipeline(featurizer, SVC())
multiclass_svm = make_pipeline(featurizer, SVC(decision_function_shape='ovr'))
xgb = make_pipeline(featurizer, XGBClassifier())
mlp = make_pipeline(featurizer, MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000))
decision_tree = make_pipeline(featurizer, DecisionTreeClassifier())

models = {
    "Dummy": dummy,
    "RandomForest": rf,
    "SVM": svm,
    "Multiclass SVM": multiclass_svm,
    "XGBoost": xgb,
    "MLP": mlp,
    "DecisionTree": decision_tree
}



## Evaluate 

In [36]:
# def evaluate_model(model, X, y):
#     """
#     Evaluate the model's performance
#     """
#     model.fit(X, y)
#     y_probs = model.predict_proba(X)[:, 1]
#     precision = precision_score(y, model.predict(X))
#     recall = recall_score(y, model.predict(X))
#     f1 = f1_score(y, model.predict(X))
#     precision_curve, recall_curve, _ = precision_recall_curve(y, y_probs)
#     auc_pr = auc(recall_curve, precision_curve)
#     cm = confusion_matrix(y, model.predict(X))
    
#     logging.info(f"Precision: {precision}")
#     logging.info(f"Recall: {recall}")
#     logging.info(f"F1-score: {f1}")
#     logging.info(f"AUC-PR score: {auc_pr}")
#     logging.info(f"Confusion Matrix:")
#     logging.info(cm)

## Grid search for Random Forest 

In [27]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

# Define the parameter grid
param_grid = {
    'n_estimators': [100, 200, 300, 400],
    'max_depth': [None, 5, 10, 15, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_leaf_nodes': [None, 10, 20, 30],
    'min_impurity_decrease': [0.0, 0.1, 0.01]
}

# Instantiate the random forest classifier
rf = RandomForestClassifier(random_state=0)

# Create the grid search object
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Fit the grid search to the training data
grid_search.fit(train.drop('VVR_group', axis=1), train['VVR_group'].values)

# Print the best parameters found
print("Best parameters found: ", grid_search.best_params_)

# Use the best estimator to make predictions
best_rf = grid_search.best_estimator_

# Predict on the validation set using the best estimator
pred = best_rf.predict(val.drop('VVR_group', axis=1))

# Calculate metrics
accuracy = accuracy_score(val['VVR_group'].values, pred)
report = classification_report(val['VVR_group'].values, pred)
precision, recall, _ = precision_recall_curve(val['VVR_group'].values, pred)
auc_pr = auc(recall, precision)

# Print results
print(f"Accuracy on Validation Data: {accuracy}")
print(f"AUC-PR on Validation Data: {auc_pr}")
print("Classification Report:")
print(report)


/Users/dionnespaltman/anaconda3/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:547: FitFailedWarning: 
10800 fits failed out of a total of 32400.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5304 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/dionnespaltman/anaconda3/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/dionnespaltman/anaconda3/lib/python3.10/site-packages/sklearn/base.py", line 1467, in wrapper
    estimator._validate_params()
  File "/Users/dionnespaltman/anaconda3/lib/python3.10/site-packages/sklearn/base.py", line 666, in _validate_params
    val

Best parameters found:  {'max_depth': 5, 'max_features': 'log2', 'max_leaf_nodes': 20, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 100}
Accuracy on Validation Data: 0.8235294117647058
AUC-PR on Validation Data: 0.8623949579831932
Classification Report:
              precision    recall  f1-score   support

           0       0.80      0.89      0.84        18
           1       0.86      0.75      0.80        16

    accuracy                           0.82        34
   macro avg       0.83      0.82      0.82        34
weighted avg       0.83      0.82      0.82        34



## Main

In [26]:
# Set logging level to INFO
logging.getLogger().setLevel(logging.INFO)

# merged_df = load_data()
# train, val, test = split_data(merged_df)
# columns_action_units = load_columns()
# featurizer = define_featurizer(columns_action_units)
# models = define_models(featurizer)

# logging.info("Applying SMOTE to handle class imbalance")
# sm = SMOTE(random_state=42)
# X_res, y_res = sm.fit_resample(train.drop('VVR_group', axis=1), train['VVR_group'].values)
# logging.info("X_res, y_res:", X_res.shape, y_res.shape)
# train_resampled = pd.DataFrame(X_res, columns=train.drop('VVR_group', axis=1).columns)
# train_resampled['VVR_group'] = y_res

logging.info("Fitting models")

for name, model in models.items():
    model.fit(train.drop('VVR_group', axis=1), train['VVR_group'].values)
    logging.info(f"Evaluating {name} on validation data")
    pred = model.predict(val.drop('VVR_group', axis=1))
    accuracy = accuracy_score(val['VVR_group'].values, pred)
    report = classification_report(val['VVR_group'].values, pred)
    
    # Calculating Precision-Recall curve and its AUC
    precision, recall, _ = precision_recall_curve(val['VVR_group'].values, pred)
    auc_pr = auc(recall, precision)

    logging.info(f"{name} Accuracy: {accuracy}")
    logging.info(f"{name} AUC-PR: {auc_pr}")  # Including AUC-PR in logging
    logging.info(f"{name} Classification Report:")
    logging.info(report)
    logging.info(f"{name} Confusion Matrix:")
    logging.info(cm)

best_model_name = max(models, key=lambda x: accuracy_score(val['VVR_group'].values, models[x].predict(val.drop('VVR_group', axis=1))))
best_model = models[best_model_name]

logging.info(f"Predicting on test using best model: {best_model_name}")

pred = best_model.predict(test.drop('VVR_group', axis=1))

accuracy = accuracy_score(test['VVR_group'].values, pred)
report = classification_report(test['VVR_group'].values, pred)
cm = confusion_matrix(test['VVR_group'].values, pred)

logging.info(f"{best_model_name} Accuracy on Test Data: {accuracy}")
logging.info(f"{best_model_name} Classification Report on Test Data:")
logging.info(report)
logging.info(f"{best_model_name} Confusion Matrix:")
logging.info(cm)


INFO:root:Fitting models


INFO:root:Evaluating Dummy on validation data
/Users/dionnespaltman/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/dionnespaltman/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/dionnespaltman/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(a

## ... 

## ... 